In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Conv2D, Lambda, MaxPool2D ,AveragePooling2D
from keras.layers import Dense, Dropout, Flatten 
from keras.layers import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator

from keras.utils.np_utils import to_categorical

In [3]:
train,test,sub = pd.read_csv('../input/digit-recognizer/train.csv'), pd.read_csv('../input/digit-recognizer/test.csv'),pd.read_csv('../input/digit-recognizer/sample_submission.csv')

In [4]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
x_train,y_train=train.drop('label',axis=1),train['label']

In [6]:
x_train/=255.0
test/=255.0

In [7]:
x_train = x_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [8]:
y_train = to_categorical(y_train, num_classes = 10)

In [9]:
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, shuffle=True,random_state=42)


In [10]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((37800, 28, 28, 1), (4200, 28, 28, 1), (37800, 10), (4200, 10))

In [11]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.01, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)


In [28]:
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same',  activation ='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(10, activation = "softmax"))
    
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
rp = keras.callbacks.ReduceLROnPlateau(
        monitor="val_acc",
        factor=0.2,
        patience=3,
        verbose=1,
        mode="max",
        min_lr=0.000001,
     )
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=100), steps_per_epoch=len(X_train)/100, 
                    epochs=30, validation_data=(X_val, y_val), callbacks=[rp])

Epoch 1/30


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.
2023-02-27 16:01:33.782682: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/dropout_4/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


378/378 [==============================] - 17s 34ms/step - loss: 0.2949 - accuracy: 0.9083 - val_loss: 2.3860 - val_accuracy: 0.4326 - lr: 0.0010
Epoch 2/30
378/378 [==============================] - 13s 34ms/step - loss: 0.0908 - accuracy: 0.9720 - val_loss: 0.0738 - val_accuracy: 0.9771 - lr: 0.0010
Epoch 3/30
378/378 [==============================] - 13s 34ms/step - loss: 0.0670 - accuracy: 0.9794 - val_loss: 0.0442 - val_accuracy: 0.9886 - lr: 0.0010
Epoch 4/30
378/378 [==============================] - 13s 34ms/step - loss: 0.0595 - accuracy: 0.9817 - val_loss: 0.0581 - val_accuracy: 0.9798 - lr: 0.0010
Epoch 5/30
378/378 [==============================] - 13s 34ms/step - loss: 0.0534 - accuracy: 0.9839 - val_loss: 0.0699 - val_accuracy: 0.9779 - lr: 0.0010
Epoch 6/30
378/378 [==============================] - 13s 36ms/step - loss: 0.0481 - accuracy: 0.9851 - val_loss: 0.0336 - val_accuracy: 0.9890 - lr: 0.0010
Epoch 7/30
378/378 [==============================] - 13s 34ms/step -

In [30]:
X_test = np.array(test)
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
Y_test = model.predict(X_test)

875/875 [==============================] - 2s 2ms/step


In [31]:
Y_test = np.argmax(Y_test, axis=1)
sub['Label']=Y_test
sub.head(10)

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


In [33]:
sub.to_csv('submission.csv', index=False)